In [9]:
df=pd.read_csv('/Users/dylanli/Downloads/RxNorm_weekly_prescribe_07162025/rrf/RXNCONSO.RRF', engine='python', sep = "|", header=None)
df.to_csv('Database/rxnconso.csv', index=False, header=False)
print(df.columns)

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], dtype='int64')


In [1]:
import pandas as pd

# Step 1: 加载 CSV 文件
conso_cols = [
    "RXCUI", "LAT", "TS", "LUI", "STT", "SUI", "ISPREF", "RXAUI", "SAUI", "SCUI", "SDUI",
    "SAB", "TTY", "CODE", "STR", "SRL", "SUPPRESS", "CVF"
]
sat_cols = [
    "RXCUI", "LUI", "SUI", "RXAUI", "STYPE", "CODE", "ATUI", "SATUI",
    "ATN", "SAB", "ATV", "SUPPRESS", "CVF"
]

rxnconso = pd.read_csv("/Users/dylanli/Downloads/RxNorm_weekly_prescribe_07162025/rrf/RXNCONSO.RRF", dtype=str, usecols=conso_cols)
rxnsat = pd.read_csv("/Users/dylanli/Downloads/RxNorm_weekly_prescribe_07162025/rrf/RXNSAT.RRF", dtype=str, usecols=sat_cols)

# Step 2: 过滤只保留英文、首选名称的条目
rxnconso = rxnconso[(rxnconso["LAT"] == "ENG") & (rxnconso["ISPREF"] == "Y")]
rxnconso = rxnconso[["RXCUI", "STR", "TTY", "CODE", "SAB"]].drop_duplicates()

# Step 3: 筛选属性说明（说明性文字）
# 如果你希望放宽条件，注释掉下一行
rxnsat = rxnsat[rxnsat["ATN"].str.lower().str.contains("description|display|text", na=False)]
rxnsat = rxnsat[["RXCUI", "ATN", "ATV"]].drop_duplicates()

# Step 4: 合并两个表
kb = pd.merge(rxnconso, rxnsat, on="RXCUI", how="left")

# Step 5: 每个药品一个条目（group by RXCUI）
kb_final = kb.groupby("RXCUI").agg({
    "STR": "first",  # 药品名
    "TTY": "first",  # 术语类型
    "CODE": "first", # 源代码
    "SAB": "first",  # 数据源
    "ATV": lambda x: " | ".join(x.dropna().unique())  # 多个描述合并
}).reset_index()

# Step 6: 保存输出
kb_final.to_csv("Database/rxnorm_knowledge_base.csv", index=False)
print("Medical Knowledge base completed as rxnorm_knowledge_base.csv")


ValueError: Usecols do not match columns, columns expected but not found: ['CODE', 'SAUI', 'SRL', 'CVF', 'SUPPRESS', 'RXAUI', 'LUI', 'LAT', 'ISPREF', 'SCUI', 'SDUI', 'SAB', 'STR', 'TTY', 'SUI', 'TS', 'RXCUI', 'STT']